목표 : 8/7~11/3 strike point
---------------

import modules

In [1]:
import pandas as pd
import numpy as np
import math
import glob
import datetime
import warnings
warnings.filterwarnings('ignore')
import os
current_directory = os.getcwd()
print("현재 디렉토리:", current_directory)

현재 디렉토리: c:\apps\Project1


load file

In [3]:
ks200 = pd.read_csv('../dataset/data/KOSPI200.csv', index_col=0)

In [ ]:
ks200

,종가,시가,고가,저가,거래량,변동 %
날짜,,,,,,
2023- 10- 31,306.33,310.52,311.51,305.81,69.93K,-0.85%
2023- 10- 30,308.95,307.10,310.09,307.01,95.85K,0.14%
2023- 10- 27,308.53,309.68,310.43,307.24,92.93M,0.25%
2023- 10- 26,307.75,311.36,312.37,307.75,110.77M,-2.76%
2023- 10- 25,316.49,319.26,319.59,316.30,91.22M,-0.61%
...,...,...,...,...,...,...
2022- 10- 19,291.29,293.41,295.34,290.90,110.85M,-0.78%
2022- 10- 18,293.59,293.34,293.78,289.77,117.12M,1.39%
2022- 10- 17,289.57,285.89,290.04,284.91,111.60M,0.07%


In [4]:
ks200.info()

<class 'pandas.core.frame.DataFrame'>
Index: 266 entries, 2023- 11- 08 to 2022- 10- 13
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   종가      266 non-null    float64
 1   시가      266 non-null    float64
 2   고가      266 non-null    float64
 3   저가      266 non-null    float64
 4   거래량     266 non-null    object 
 5   변동 %    266 non-null    object 
dtypes: float64(4), object(2)
memory usage: 14.5+ KB


In [5]:
# 음수 값이 있을수 있으므로 절댓값 취하기
ks200 = ks200['종가'].abs()

In [6]:
ks200.index = pd.to_datetime(ks200.index, format="%Y- %m- %d")

In [43]:
# 날짜 7월 31일부터 -> 옵션은 전날 종가로 기준 행사가를 잡음
# ks200 = ks200.loc[ks200['일자'] > 20230727]

In [7]:
ks200.head(3)

날짜
2023-11-08    325.59
2023-11-07    327.29
2023-11-06    334.16
Name: 종가, dtype: float64

##### make dataframe strike price

In [8]:
df = ks200.copy()

In [9]:
df

날짜
2023-11-08    325.59
2023-11-07    327.29
2023-11-06    334.16
2023-11-03    318.24
2023-11-02    315.16
               ...  
2022-10-19    291.29
2022-10-18    293.59
2022-10-17    289.57
2022-10-14    289.36
2022-10-13    282.57
Name: 종가, Length: 266, dtype: float64

In [11]:
# 소수점만 가져오기
result = df.apply(lambda x: x % 10)

In [12]:
result

날짜
2023-11-08    5.59
2023-11-07    7.29
2023-11-06    4.16
2023-11-03    8.24
2023-11-02    5.16
              ... 
2022-10-19    1.29
2022-10-18    3.59
2022-10-17    9.57
2022-10-14    9.36
2022-10-13    2.57
Name: 종가, Length: 266, dtype: float64

In [13]:
result = pd.DataFrame(result)
result.columns = ['소수']

In [14]:
result

,소수
날짜,
2023-11-08,5.59
2023-11-07,7.29
2023-11-06,4.16
2023-11-03,8.24
2023-11-02,5.16
...,...
2022-10-19,1.29
2022-10-18,3.59
2022-10-17,9.57


In [15]:
price = pd.concat([df, result], axis=1)

In [16]:
price.head(3)

,종가,소수
날짜,,
2023-11-08,325.59,5.59
2023-11-07,327.29,7.29
2023-11-06,334.16,4.16


In [17]:
# 행사가 컬럼 생성
price['행사가'] = 0

In [18]:
# 소수에 따라 기준 행사가 지정
for index, i in enumerate(price['소수']):
	if i > 7.5:
		price['행사가'][index] = 7.5
	elif i > 5:
		price['행사가'][index] = 5
	elif i > 2.5:
		price['행사가'][index] = 2.5
	else:
		price['행사가'][index] = 0

In [19]:
price

,종가,소수,행사가
날짜,,,
2023-11-08,325.59,5.59,5.0
2023-11-07,327.29,7.29,5.0
2023-11-06,334.16,4.16,2.5
2023-11-03,318.24,8.24,7.5
2023-11-02,315.16,5.16,5.0
...,...,...,...
2022-10-19,291.29,1.29,0.0
2022-10-18,293.59,3.59,2.5
2022-10-17,289.57,9.57,7.5


In [20]:
price['기준행사가'] = price['행사가'] + (price['종가'].apply(lambda x: x // 10) * 10)

In [21]:
price

,종가,소수,행사가,기준행사가
날짜,,,,
2023-11-08,325.59,5.59,5.0,325.0
2023-11-07,327.29,7.29,5.0,325.0
2023-11-06,334.16,4.16,2.5,332.5
2023-11-03,318.24,8.24,7.5,317.5
2023-11-02,315.16,5.16,5.0,315.0
...,...,...,...,...
2022-10-19,291.29,1.29,0.0,290.0
2022-10-18,293.59,3.59,2.5,292.5
2022-10-17,289.57,9.57,7.5,287.5


### 기준행사가

In [22]:
strike_price = price[['기준행사가']]

기준행사가 및 (+-4)

In [23]:
strike_price['S-4'] = strike_price['기준행사가'] - 10
strike_price['S-3'] = strike_price['기준행사가'] - 7.5
strike_price['S-2'] = strike_price['기준행사가'] - 5
strike_price['S-1'] = strike_price['기준행사가'] - 2.5
strike_price['S+1'] = strike_price['기준행사가'] + 2.5
strike_price['S+2'] = strike_price['기준행사가'] + 5
strike_price['S+3'] = strike_price['기준행사가'] + 7.5
strike_price['S+4'] = strike_price['기준행사가'] + 10

In [24]:
strike_price

,기준행사가,S-4,S-3,S-2,S-1,S+1,S+2,S+3,S+4
날짜,,,,,,,,,
2023-11-08,325.0,315.0,317.5,320.0,322.5,327.5,330.0,332.5,335.0
2023-11-07,325.0,315.0,317.5,320.0,322.5,327.5,330.0,332.5,335.0
2023-11-06,332.5,322.5,325.0,327.5,330.0,335.0,337.5,340.0,342.5
2023-11-03,317.5,307.5,310.0,312.5,315.0,320.0,322.5,325.0,327.5
2023-11-02,315.0,305.0,307.5,310.0,312.5,317.5,320.0,322.5,325.0
...,...,...,...,...,...,...,...,...,...
2022-10-19,290.0,280.0,282.5,285.0,287.5,292.5,295.0,297.5,300.0
2022-10-18,292.5,282.5,285.0,287.5,290.0,295.0,297.5,300.0,302.5
2022-10-17,287.5,277.5,280.0,282.5,285.0,290.0,292.5,295.0,297.5


In [25]:
strike_price.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 266 entries, 2023-11-08 to 2022-10-13
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   기준행사가   266 non-null    float64
 1   S-4     266 non-null    float64
 2   S-3     266 non-null    float64
 3   S-2     266 non-null    float64
 4   S-1     266 non-null    float64
 5   S+1     266 non-null    float64
 6   S+2     266 non-null    float64
 7   S+3     266 non-null    float64
 8   S+4     266 non-null    float64
dtypes: float64(9)
memory usage: 20.8 KB


In [26]:
strike_price.head(33)

,기준행사가,S-4,S-3,S-2,S-1,S+1,S+2,S+3,S+4
날짜,,,,,,,,,
2023-11-08,325.0,315.0,317.5,320.0,322.5,327.5,330.0,332.5,335.0
2023-11-07,325.0,315.0,317.5,320.0,322.5,327.5,330.0,332.5,335.0
2023-11-06,332.5,322.5,325.0,327.5,330.0,335.0,337.5,340.0,342.5
2023-11-03,317.5,307.5,310.0,312.5,315.0,320.0,322.5,325.0,327.5
2023-11-02,315.0,305.0,307.5,310.0,312.5,317.5,320.0,322.5,325.0
2023-11-01,307.5,297.5,300.0,302.5,305.0,310.0,312.5,315.0,317.5
2023-10-31,305.0,295.0,297.5,300.0,302.5,307.5,310.0,312.5,315.0
2023-10-30,307.5,297.5,300.0,302.5,305.0,310.0,312.5,315.0,317.5
2023-10-27,307.5,297.5,300.0,302.5,305.0,310.0,312.5,315.0,317.5


In [62]:
# strike_price = strike_price.set_index('일자')

In [63]:
# 전날 종가가 당일 기준행사가이므로 행 밀어주기
strike_price = strike_price.shift(-1)

In [64]:
strike_price.head(22)

,기준행사가,S-4,S-3,S-2,S-1,S+1,S+2,S+3,S+4
날짜,,,,,,,,,
2023-10-31,307.5,297.5,300.0,302.5,305.0,310.0,312.5,315.0,317.5
2023-10-30,307.5,297.5,300.0,302.5,305.0,310.0,312.5,315.0,317.5
2023-10-27,307.5,297.5,300.0,302.5,305.0,310.0,312.5,315.0,317.5
2023-10-26,315.0,305.0,307.5,310.0,312.5,317.5,320.0,322.5,325.0
2023-10-25,317.5,307.5,310.0,312.5,315.0,320.0,322.5,325.0,327.5
2023-10-24,315.0,305.0,307.5,310.0,312.5,317.5,320.0,322.5,325.0
2023-10-23,317.5,307.5,310.0,312.5,315.0,320.0,322.5,325.0,327.5
2023-10-20,322.5,312.5,315.0,317.5,320.0,325.0,327.5,330.0,332.5
2023-10-19,327.5,317.5,320.0,322.5,325.0,330.0,332.5,335.0,337.5


In [65]:
# strike_price.drop(index=[20230728, 20231023, 20231024], axis=0, inplace=True)

In [27]:
strike_price.index

DatetimeIndex(['2023-11-08', '2023-11-07', '2023-11-06', '2023-11-03',
               '2023-11-02', '2023-11-01', '2023-10-31', '2023-10-30',
               '2023-10-27', '2023-10-26',
               ...
               '2022-10-26', '2022-10-25', '2022-10-24', '2022-10-21',
               '2022-10-20', '2022-10-19', '2022-10-18', '2022-10-17',
               '2022-10-14', '2022-10-13'],
              dtype='datetime64[ns]', name='날짜', length=266, freq=None)

# 기간조정

In [28]:
strike_price.index

DatetimeIndex(['2023-11-08', '2023-11-07', '2023-11-06', '2023-11-03',
               '2023-11-02', '2023-11-01', '2023-10-31', '2023-10-30',
               '2023-10-27', '2023-10-26',
               ...
               '2022-10-26', '2022-10-25', '2022-10-24', '2022-10-21',
               '2022-10-20', '2022-10-19', '2022-10-18', '2022-10-17',
               '2022-10-14', '2022-10-13'],
              dtype='datetime64[ns]', name='날짜', length=266, freq=None)

In [29]:
start_date = pd.to_datetime("2023-10-10")
end_date = pd.to_datetime("2023-11-08")
strike_price = strike_price.loc[end_date:start_date]

# to_csv

In [31]:
strike_price.to_csv('../dataset/strike_price.csv')

In [30]:
strike_price

,기준행사가,S-4,S-3,S-2,S-1,S+1,S+2,S+3,S+4
날짜,,,,,,,,,
2023-11-08,325.0,315.0,317.5,320.0,322.5,327.5,330.0,332.5,335.0
2023-11-07,325.0,315.0,317.5,320.0,322.5,327.5,330.0,332.5,335.0
2023-11-06,332.5,322.5,325.0,327.5,330.0,335.0,337.5,340.0,342.5
2023-11-03,317.5,307.5,310.0,312.5,315.0,320.0,322.5,325.0,327.5
2023-11-02,315.0,305.0,307.5,310.0,312.5,317.5,320.0,322.5,325.0
2023-11-01,307.5,297.5,300.0,302.5,305.0,310.0,312.5,315.0,317.5
2023-10-31,305.0,295.0,297.5,300.0,302.5,307.5,310.0,312.5,315.0
2023-10-30,307.5,297.5,300.0,302.5,305.0,310.0,312.5,315.0,317.5
2023-10-27,307.5,297.5,300.0,302.5,305.0,310.0,312.5,315.0,317.5


In [71]:
# oct = strike_price[:9]
# sep = strike_price[9:25]
# aug = strike_price[25:44]
# july = strike_price[44:]

In [72]:
# 해당 달 별 최소 행사가와 최대 행사가 찾기
# oct.describe()